In [ ]:
# Copyright (c) 2012-2023, NECOTIS
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without modification,
# are permitted provided that the following conditions are met:
#
#  - Redistributions of source code must retain the above copyright notice,
#    this list of conditions and the following disclaimer.
#  - Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.
#  - Neither the name of the copyright holder nor the names of its contributors
#    may be used to endorse or promote products derived from this software
#    without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
# ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
# WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED.
# IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT,
# INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT
# NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA,
# OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
# WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.

# Authors: Emmanuel Calvet, Jean Rouat, Bertrand Reulet (advisor)
# Date: July 07th, 2023
# Organization: Groupe de recherche en Neurosciences Computationnelles et Traitement Intelligent des Signaux (NECOTIS),
# Université de Sherbrooke, Canada

In [ ]:
# Import library and models
# NB: you need to import binary_model into the python path
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Standard library
import numpy as np
import matplotlib.pyplot as plt
#!pip install tqdm 
from tqdm import tqdm

# The model
from model.binaryModel import binaryModel
from model.utils import initRandomArchitecture, initGaussianWeights, initUniformWeights, dataPath

In [ ]:
# The experiment
sim = 'free_evolution'
distribution_weight = 'normal'
experiment = 'display_phase_transition'
experiment = experiment + '_' + distribution_weight

N = 10000
duration = 2000
nb_reservoir = 100 # Number of reservoir per sigma value (each is ran once)
K = 16 # Connectivity degree of the graph 
meanWeight = -0.1 # Average weights for the normal distribution
# List of standard deviation sigma (of the normal distribution)
nb_sigmas = 80
exponents = np.linspace(-2, 2, nb_sigmas)
sigmas = np.power(10, exponents)

print(f'The model will be evaluated on {len(sigmas)} values of sigma(W):')
print('sigma(W) =', sigmas)

In [ ]:
# Create the model
optionData = {'spikeCount':True, 'indexActive':False} # Option for saving data
brain = binaryModel(N, sim=sim, experiment=experiment)
seedConnectivity = 747
initRandomArchitecture(brain, K, seed=seedConnectivity) # Fixed architecture

In [ ]:
# Function to check if reservoir is ran already

def check_if_ran(sim, experiment, N, K, meanWeight, sigma):
    #Check network has not been run already
    pathBrain = dataPath(sim, experiment, N, K, meanWeight)
    file_path = pathBrain + f"/metadata_N{N}_K{K}_T{nb_reservoir}_W{meanWeight}_std{sigma}.npy"
    if os.path.exists(file_path):
        print('Network already run, next...\n \n')
        return True
    else:
        return False

In [ ]:
# Run the model

for sigma in tqdm(sigmas):
    sigma = np.round(sigma, 5)
    optionData = {'spikeCount':True, 'indexActive':False}
    for t in range(nb_reservoir):
        
        # Check if the network is already run
        check = check_if_ran(sim, experiment, N, K, meanWeight, sigma)
        if check:
            continue
        
        # Select weight distribution
        if distribution_weight == 'normal':
            initGaussianWeights(brain, meanWeight, sigma, seed=t*100)
        elif distribution_weight == 'uniform':
            initUniformWeights(brain, meanWeight, sigma, seed=t*100)
        
        # Run simulation
        brain.displayInfoNetwork()
        brain.initRandomState(0.2, seed=t*100) # Initialize the state of the reservoir
        brain.run(duration, **optionData)
        
        # Reset the network state
        brain.reset()
    
    # Save data and metadata
    brain.saveData(**optionData)
    brain.saveMetadata()
    brain.reset(deep=True)


In [ ]:
# Analyse the data

from model.utils import statisticsActivity
import glob 

def statisticActivity_over_sigma(meanWeight, N, K, nb_reservoir,
                                 sim='free_evolution', experiment='display_phase_transition'):
       
    result_dir = dataPath(sim, experiment)
    analysis_folder = os.path.join(result_dir, sim)
    if not os.path.isdir(analysis_folder):
        os.makedirs(analysis_folder)
        
    # Figure
    avgLabel = r'$<<A>_t>_T}$'
    varLabel = r'$<\sigma^2_t(A)/N>_T$'
    xlabel = r'$|\sigma^{\star}|$'
    fig = plt.figure()
    ax = fig.add_subplot(2, 1, 1)
    ax.set_ylabel(avgLabel)
    ax.set_title('Average')
    ax1 = fig.add_subplot(2, 1, 2)
    ax1.set_xlabel(xlabel)
    ax1.set_ylabel(varLabel)
    ax1.set_title('Variance')
    
    avgAll = []
    varAll = []

    # data folder
    directory = dataPath(sim, experiment, N, K, meanWeights=meanWeight)
    filenames = glob.glob(directory + '/metadata_*.npy'.format(nb_reservoir))
    stdWeights = []
    for filename in filenames:

        # Load datas
        metadata = np.load(filename, allow_pickle='TRUE').item()
        directoryActivity = metadata['dataPath']
        stdWeights.append(metadata['stdWeights'])
        fileData = metadata['spikeCountFile']
        duration = metadata['duration']
        idxStart = int(duration / 2)
        # Load activity data
        filepath = directoryActivity + '/' + fileData + '.npy'
        print(filepath)
        A_trials = np.load(filepath)

        # Global statistics
        avgTrials, varTrials = statisticsActivity(N, A_trials, idxStart)
        avgAll.append(avgTrials)
        varAll.append(varTrials)

    ## plot average of all trials
    stdWeights = np.array(stdWeights)
    order = np.argsort(stdWeights)
    stdWeights = stdWeights[order]
    avgAll = np.array(avgAll)
    avgAll = avgAll[order]
    varAll = np.array(varAll)
    varAll = varAll[order]

    if meanWeight>0:
        marker = '^-'
        label = '$\sigma^{\star}>0$'
    elif meanWeight<0:
        marker = 'v-'
        label = '$\sigma^{\star}<0$'
    else:
        marker = 'o-'
        label = '$\mu=0$'
    
    if meanWeight != 0:
        x_axis = np.abs(stdWeights/meanWeight)
    else:
        x_axis = np.abs(stdWeights)

    ax.semilogx(x_axis, avgAll, marker, label=label)
    color = ax.get_lines()[-1].get_color()
    ax1.semilogx(x_axis, varAll, marker, color=color)

    plt.tight_layout()
    fig.savefig(analysis_folder + f'statActivity_sorted_N{N}_K{K}_d{duration}_T{nb_reservoir}.jpg')


In [ ]:
# Run analysis of activity statistics
statisticActivity_over_sigma(meanWeight, N, K, nb_reservoir,
                             sim=sim, experiment=experiment)